In [1]:
import tensorflow as tf
from glob import glob
from tqdm import tqdm
import os

2024-10-26 14:57:52.844027: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-26 14:57:52.985891: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-26 14:57:53.028307: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-26 14:57:53.067713: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-26 14:57:55.235559: W tensorflow/compiler/tf2

In [2]:

blured_final_dir = "../data/finalDataset/bluredImageDataset"
keypoint_final_dir = "../data/finalDataset/keypointDataset"
class_names = sorted([d for d in os.listdir(blured_final_dir+"/train") if os.path.isdir(os.path.join(blured_final_dir+"/train", d))])
class_dict = {x:i for i,x in enumerate(class_names)}
num_classes = len(class_names)
num_classes, class_names, class_dict

(2, ['class_01', 'class_02'], {'class_01': 0, 'class_02': 1})

In [3]:
import numpy as np
def get_labels(image_dirs):
    labels = []
    for x in image_dirs:
        labels.append(class_dict[x.split("/")[-2]])
    return np.array(labels)

In [4]:
X_train_paths_1 = sorted(glob(blured_final_dir+"/train/*/*.npy"))
X_train_paths_2 = sorted(glob(keypoint_final_dir+"/train/*/*.npy"))
train_labels = get_labels(X_train_paths_1)

len(X_train_paths_1), len(X_train_paths_2), X_train_paths_1[0], X_train_paths_2[0]

(112,
 112,
 '../data/finalDataset/bluredImageDataset/train/class_01/00000_bai-yao-pu-episode-3-720.npy',
 '../data/finalDataset/keypointDataset/train/class_01/00000_bai-yao-pu-episode-3-720.npy')

In [5]:
X_test_paths_1 = sorted(glob(blured_final_dir+"/test/*/*.npy"))
X_test_paths_2 = sorted(glob(keypoint_final_dir+"/test/*/*.npy"))
test_labels = get_labels(X_test_paths_1)

len(X_test_paths_1), len(X_test_paths_2), X_test_paths_1[0], X_test_paths_2[0]

(2,
 2,
 '../data/finalDataset/bluredImageDataset/test/class_01/00004_bai-yao-pu-episode-5-720.npy',
 '../data/finalDataset/keypointDataset/test/class_01/00004_bai-yao-pu-episode-5-720.npy')

In [6]:
for x in X_test_paths_1:
    x_data = np.load(x)
    assert x_data.shape == (20, 2048)
for x in X_test_paths_1:
    x_data = np.load(x)
    assert x_data.shape == (20, 2048)

for x in X_train_paths_2:
    x_data = np.load(x)
    assert x_data.shape == (20, 102)

for x in X_test_paths_2:
    x_data = np.load(x)
    assert x_data.shape == (20, 102)

In [7]:

y_train = tf.one_hot(train_labels, depth=num_classes)
y_test = tf.one_hot(test_labels, depth=num_classes)

In [8]:
def process_path(input_1_path, input_2_path, label):
    # Define helper to load .npy files
    def _process_npy_file(file_path):
        # Load .npy file and return tensor
        data = np.load(file_path.decode("utf-8"))  # decode the file path to string
        return tf.convert_to_tensor(data, dtype=tf.float32)
    
    # Apply `tf.numpy_function` to load each input separately
    input_1 = tf.numpy_function(_process_npy_file, [input_1_path], tf.float32)
    input_2 = tf.numpy_function(_process_npy_file, [input_2_path], tf.float32)
    
    return (input_1, input_2), label


In [9]:
BATCH_SIZE = 32
train_ds = tf.data.Dataset.from_tensor_slices(((X_train_paths_1, X_train_paths_2), y_train))
test_ds = tf.data.Dataset.from_tensor_slices(((X_test_paths_1, X_test_paths_2), y_test))

train_ds = train_ds.map(lambda x, y: process_path(x[0], x[1], y), num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.map(lambda x, y: process_path(x[0], x[1], y), num_parallel_calls=tf.data.AUTOTUNE)

train_ds = train_ds.batch(BATCH_SIZE)
test_ds = test_ds.batch(BATCH_SIZE)


In [10]:
# Check shapes in a batch from train_ds
for batch in train_ds.take(1):
    inputs, labels = batch
    input_1, input_2 = inputs
    print("Input 1 shape:", input_1.shape)
    print("Input 2 shape:", input_2.shape)
    print("Label shape:", labels.shape)


Input 1 shape: (32, 20, 2048)
Input 2 shape: (32, 20, 102)
Label shape: (32, 2)


2024-10-26 14:58:07.943423: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [59]:
for input_tensor in input_shapes:
    print(input_tensor.shape)


(20, 2048)
(20, 102)


In [16]:
import tensorflow as tf
import numpy as np

BATCH_SIZE = 32

def load_npy_files(filePath):
    # Load .npy file and return tensor
    data = np.load(filePath.decode("utf-8"))  # decode the file path to string
    return tf.convert_to_tensor(data, dtype=tf.float32)

def create_dataset(mesh_files, keypoints_files, labels, batch_size):
    def load_data(mesh_file, keypoints_file, label):
        # Load mesh and keypoints data from files
        mesh_data = tf.numpy_function(load_npy_files, [mesh_file], tf.float32)  
        keypoints_data = tf.numpy_function(load_npy_files, [keypoints_file], tf.float32)    
        # Set the shape explicitly after loading the data
        mesh_data.set_shape((20, 2048))  # Assuming your mesh data has this shape
        keypoints_data.set_shape((20, 102)) # Assuming your keypoints data has this shape
        return (mesh_data, keypoints_data), label

    # Create a dataset from file paths and labels
    dataset = tf.data.Dataset.from_tensor_slices((mesh_files, keypoints_files, labels))
    dataset = dataset.map(load_data, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

# Create train and test datasets
train_dataset = create_dataset(X_train_paths_1, X_train_paths_2, y_train, BATCH_SIZE)
test_dataset = create_dataset(X_test_paths_1, X_test_paths_2, y_test, BATCH_SIZE)

In [17]:
for batch_idx, ((mesh_data, keypoints_data), labels) in enumerate(train_dataset):
    # The shapes will be printed during execution
    print("Mesh data shape:", mesh_data.shape)
    print("Keypoints data shape:", keypoints_data.shape)
    print("Labels shape:", labels.shape)
    print()

    # Your model training code here

    if batch_idx >= 2:  # Print shapes for the first 3 batches
        break

    if batch_idx >= 2:  # Print shapes for the first 3 batches
        break

Mesh data shape: (32, 20, 2048)
Keypoints data shape: (32, 20, 102)
Labels shape: (32, 2)

Mesh data shape: (32, 20, 2048)
Keypoints data shape: (32, 20, 102)
Labels shape: (32, 2)

Mesh data shape: (32, 20, 2048)
Keypoints data shape: (32, 20, 102)
Labels shape: (32, 2)



In [18]:
tf.random.set_seed(42)
from tensorflow.keras import Sequential, Input, Model
from tensorflow.keras.layers import LSTM, Dense, GRU,Dropout, concatenate

# # Define the keypoint model
# model_keypoint = Sequential()
# model_keypoint.add(LSTM(64, return_sequences=True, activation='relu', input_shape=train_keypoints_dataset.shape[1:]))
# model_keypoint.add(LSTM(128, return_sequences=True, activation='relu'))
# model_keypoint.add(LSTM(64, return_sequences=False, activation='relu'))
# model_keypoint.add(Dense(64, activation='relu'))
# model_keypoint.add(Dense(32, activation='relu'))

key_input = Input(shape=(20,102))
x = LSTM(64, return_sequences=True, activation='relu')(key_input)
x = LSTM(128, return_sequences=True, activation='relu')(x)
x = LSTM(64, return_sequences=False, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(32, activation='relu')(x)
key_output = x
model_keypoint = Model(inputs=key_input, outputs=key_output)

# Define the image model
img_input = Input(shape=(20,2048))
x = LSTM(64, return_sequences=True)(img_input)
x = GRU(8)(x)
x = Dropout(0.4)(x)
img_output = Dense(8, activation="relu")(x)
model_img = Model(inputs=img_input, outputs=img_output)

# Concatenate the outputs of both models
combined_output = concatenate([model_img.output, model_keypoint.output])

# Add a Dense layer for the final prediction
output = Dense(num_classes, activation="softmax")(combined_output)

# Create the combined model
model = Model(
    inputs=[
        model_img.input,
        model_keypoint.input
    ],
    outputs=output
)

model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 20, 102)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_3       │ (None, 20, 2048)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_4 (LSTM)       │ (None, 20, 64)    │     42,752 │ input_layer_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_7 (LSTM)       │ (None, 20, 64)    │    540,928 │ input_layer_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_5 (LSTM)       │ (None, 20, 128)   │     98,816 │ lstm_4[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru_1 (GRU)         │ (None, 8)         │      1,776 │ lstm_7[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_6 (LSTM)       │ (None, 64)        │     49,408 │ lstm_5[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 8)         │          0 │ gru_1[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 64)        │      4,160 │ lstm_6[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 8)         │         72 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 32)        │      2,080 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 40)        │          0 │ dense_6[0][0],    │
│ (Concatenate)       │                   │            │ dense_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 2)         │         82 │ concatenate_1[0]… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 740,074 (2.82 MB)

 Trainable params: 740,074 (2.82 MB)

 Non-trainable params: 0 (0.00 B)

In [19]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [20]:
history = model.fit(
    train_dataset,
    epochs=5,
    validation_data=test_dataset
    )

Epoch 1/5


/home/yato/ML/human-activity-recognition/env/lib/python3.12/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_19', 'keras_tensor_13']. Received: the structure of inputs=('*', '*')
  warnings.warn(


4/4 ━━━━━━━━━━━━━━━━━━━━ 21s 898ms/step - accuracy: 0.2438 - loss: 0.7458 - val_accuracy: 0.5000 - val_loss: 0.6981
Epoch 2/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 157ms/step - accuracy: 0.5433 - loss: 0.6910 - val_accuracy: 0.5000 - val_loss: 0.7068
Epoch 3/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 166ms/step - accuracy: 0.4415 - loss: 0.7276 - val_accuracy: 0.5000 - val_loss: 0.7098
Epoch 4/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 135ms/step - accuracy: 0.4787 - loss: 0.7054 - val_accuracy: 0.5000 - val_loss: 0.7140
Epoch 5/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 152ms/step - accuracy: 0.5308 - loss: 0.7026 - val_accuracy: 0.5000 - val_loss: 0.7222


[(None, 20, 2048), (None, 20, 102)]